# Backpropagation Algorithm

In [3]:
import pandas as pd
import json
import pprint as pp
import math
import copy
from IPython.display import display 
import numpy as np

In [4]:
def weighted_sum(values, weights):
    res = 0
    for value, weight in zip(values, weights):
        res += value * weight
    return res

In [5]:
def actual_output(weighted_sum): 
    return 1/(1 + math.exp(-weighted_sum))

In [6]:
def delta_ou(target_output, actual_output_ou):
    return (target_output - actual_output_ou) * actual_output_ou * (1-actual_output_ou)

In [7]:
def new_weight_to_ou(weight_to_ou, learning_coef, delta_ou, actual_output_hu):
    return weight_to_ou + learning_coef * delta_ou * actual_output_hu

In [8]:
def delta_hu(delta_ou, weight_to_ou, actual_output_hu):
        return delta_ou * weight_to_ou * actual_output_hu * (1 - actual_output_hu) 

In [9]:
def new_weight_to_hu(weight_to_hu, learning_coef, delta_hu, value_iu):
    return weight_to_hu + learning_coef * delta_hu * value_iu

In [10]:
def terminate_training(actual_outputs_of_ou, target_outputs, lower_margin, upper_margin):
    for i, target_output in enumerate(target_outputs):
        if target_output == 1:
            if actual_outputs_of_ou[i] <= upper_margin:
                return False
        else:
            if actual_outputs_of_ou[i] >= lower_margin:
                return False
    return True

In [11]:
def bpa(dataset, num_steps = 1, roundup = 4):
    log = []

    for i, inputs in enumerate(dataset['inputs_list']):
        
        weights_to_hus = copy.deepcopy(dataset['weights_to_hidden_units'])
        weights_to_ous = copy.deepcopy(dataset['weights_to_output_units'])
        target_outputs = copy.deepcopy(dataset['target_outputs_list'][i])

        sub_log = []
        step = 0
        while step <= num_steps:
            actual_output_of_hus = []
            for weights_to_hu in weights_to_hus:
                actual_output_of_hus.append(actual_output(weighted_sum(inputs, weights_to_hu)))
            
            actual_output_of_ous = []
            for weights_to_ou in weights_to_ous:
                actual_output_of_ous.append(actual_output(weighted_sum(actual_output_of_hus, weights_to_ou)))

            sub_sub_log = {}
            sub_sub_log.update({"1.step": step})
            sub_sub_log.update({"2.actual_output_of_hus": np.round(actual_output_of_hus, roundup)})
            sub_sub_log.update({"2.actual_output_of_ous": np.round(actual_output_of_ous, roundup)})
            sub_sub_log.update({"3.weights_to_ous": np.round(weights_to_ous, roundup)})
            sub_sub_log.update({"4.weights_to_hus": np.round(weights_to_hus, roundup)})
            sub_log.append(sub_sub_log)

            
            trained = terminate_training(actual_output_of_ous, target_outputs, dataset['lower_margin'], dataset['upper_margin'])
            if trained:
                # print('The training is complete.')
                break
            else:
                temp_weights_to_ous = copy.deepcopy(weights_to_ous)

                delta_ous = []
                for k, actual_output_of_ou in enumerate(actual_output_of_ous):
                    delta_ous.append(delta_ou(target_outputs[k], actual_output_of_ou))
                    
                    for j, actual_output_of_hu in enumerate(actual_output_of_hus):
                        weights_to_ous[k][j] = new_weight_to_ou(weights_to_ous[k][j], dataset['learning_coef'], delta_ous[k], actual_output_of_hu)

                # print(f'delta_ous: {delta_ous}')
                # print(f'weights_to_ous: {weights_to_ous}')

                delta_hus = []
                for j, actual_output_of_hu in enumerate(actual_output_of_hus):
                    temp_delta_hu = 0
                    for k, actual_output_of_ou in enumerate(actual_output_of_ous):
                        temp_delta_hu += delta_hu(delta_ous[k], temp_weights_to_ous[k][j], actual_output_of_hu)
                    delta_hus.append(temp_delta_hu)

                for j, actual_output_of_hu in enumerate(actual_output_of_hus):
                    for i, input in enumerate(inputs):
                        weights_to_hus[j][i] = new_weight_to_hu(weights_to_hus[j][i], dataset['learning_coef'], delta_hus[j], input)
                
                # print(f'delta_hus: {delta_hus}')
                # print(f'weights_to_hus: {weights_to_hus}')
            
            step += 1
        log.append({"inputs": inputs, "target_outputs": target_outputs, "training": sub_log})
        # break #only on input
    return log

In [12]:
with open('dataset.json', 'r') as file:
    dataset = json.load(file)

In [16]:
for data in dataset:
    pp.pprint(data)
    # https://www.geeksforgeeks.org/display-the-pandas-dataframe-in-table-style/
    log = bpa(data, 1000, 5)
    for sub_log in log:
        print(f"\n\ninputs: {sub_log['inputs']}; target_outputs: {sub_log['target_outputs']}")
        # df = pd.DataFrame(sub_log['training'][:10])
        df = pd.DataFrame(sub_log['training'][-10:])
        # displaying the DataFrame
        display(df.style)

{'inputs_list': [[0, 0, 0],
                 [1, 0, 0],
                 [0, 1, 0],
                 [1, 1, 0],
                 [0, 0, 1],
                 [1, 0, 1],
                 [0, 1, 1],
                 [1, 1, 1]],
 'learning_coef': 0.5,
 'lower_margin': 0.1,
 'target_outputs_list': [[0], [1], [1], [0], [0], [1], [1], [0]],
 'upper_margin': 0.9,
 'weights_to_hidden_units': [[0.2, 0.3, 0.4], [0.5, 0.6, 0.7]],
 'weights_to_output_units': [[0.8, 0.9]]}


inputs: [0, 0, 0]; target_outputs: [0]


,1.step,2.actual_output_of_hus,2.actual_output_of_ous,3.weights_to_ous,4.weights_to_hus
0,300,[0.5 0.5],[0.10179],[[-2.22754 -2.12754]],[[0.2 0.3 0.4] [0.5 0.6 0.7]]
1,301,[0.5 0.5],[0.10157],[[-2.22987 -2.12987]],[[0.2 0.3 0.4] [0.5 0.6 0.7]]
2,302,[0.5 0.5],[0.10136],[[-2.23218 -2.13218]],[[0.2 0.3 0.4] [0.5 0.6 0.7]]
3,303,[0.5 0.5],[0.10115],[[-2.23449 -2.13449]],[[0.2 0.3 0.4] [0.5 0.6 0.7]]
4,304,[0.5 0.5],[0.10094],[[-2.23679 -2.13679]],[[0.2 0.3 0.4] [0.5 0.6 0.7]]
5,305,[0.5 0.5],[0.10074],[[-2.23908 -2.13908]],[[0.2 0.3 0.4] [0.5 0.6 0.7]]
6,306,[0.5 0.5],[0.10053],[[-2.24136 -2.14136]],[[0.2 0.3 0.4] [0.5 0.6 0.7]]
7,307,[0.5 0.5],[0.10032],[[-2.24363 -2.14363]],[[0.2 0.3 0.4] [0.5 0.6 0.7]]
8,308,[0.5 0.5],[0.10012],[[-2.2459 -2.1459]],[[0.2 0.3 0.4] [0.5 0.6 0.7]]
9,309,[0.5 0.5],[0.09992],[[-2.24815 -2.14815]],[[0.2 0.3 0.4] [0.5 0.6 0.7]]




inputs: [1, 0, 0]; target_outputs: [1]


,1.step,2.actual_output_of_hus,2.actual_output_of_ous,3.weights_to_ous,4.weights_to_hus
0,109,[0.63128 0.70086],[0.89589],[[1.51684 1.70474]],[[0.53769 0.3 0.4 ] [0.85141 0.6 0.7 ]]
1,110,[0.63167 0.70123],[0.8964],[[1.5199 1.70814]],[[0.53941 0.3 0.4 ] [0.85314 0.6 0.7 ]]
2,111,[0.63207 0.70159],[0.89691],[[1.52294 1.71151]],[[0.54111 0.3 0.4 ] [0.85487 0.6 0.7 ]]
3,112,[0.63246 0.70194],[0.89742],[[1.52595 1.71486]],[[0.54279 0.3 0.4 ] [0.85657 0.6 0.7 ]]
4,113,[0.63285 0.7023 ],[0.89791],[[1.52894 1.71817]],[[0.54447 0.3 0.4 ] [0.85827 0.6 0.7 ]]
5,114,[0.63324 0.70265],[0.89841],[[1.5319 1.72146]],[[0.54613 0.3 0.4 ] [0.85995 0.6 0.7 ]]
6,115,[0.63362 0.703 ],[0.89889],[[1.53484 1.72472]],[[0.54778 0.3 0.4 ] [0.86162 0.6 0.7 ]]
7,116,[0.634 0.70334],[0.89937],[[1.53775 1.72795]],[[0.54942 0.3 0.4 ] [0.86327 0.6 0.7 ]]
8,117,[0.63438 0.70369],[0.89985],[[1.54064 1.73115]],[[0.55104 0.3 0.4 ] [0.86491 0.6 0.7 ]]
9,118,[0.63475 0.70403],[0.90032],[[1.5435 1.73432]],[[0.55266 0.3 0.4 ] [0.86654 0.6 0.7 ]]




inputs: [0, 1, 0]; target_outputs: [1]


...

,1.step,2.actual_output_of_hus,2.actual_output_of_ous,3.weights_to_ous,4.weights_to_hus
0,72,[0.73111 0.82541],[0.89489],[[1.28364 1.45766]],[[0.2 0.45014 0.55014] [0.5 0.72671 0.82671]]
1,73,[0.7316 0.82571],[0.89555],[[1.28725 1.46174]],[[0.2 0.45139 0.55139] [0.5 0.72775 0.82775]]
2,74,[0.73209 0.826 ],[0.8962],[[1.29083 1.46577]],[[0.2 0.45262 0.55262] [0.5 0.72878 0.82878]]
3,75,[0.73257 0.82629],[0.89685],[[1.29436 1.46976]],[[0.2 0.45384 0.55384] [0.5 0.7298 0.8298 ]]
4,76,[0.73304 0.82658],[0.89748],[[1.29786 1.4737 ]],[[0.2 0.45505 0.55505] [0.5 0.7308 0.8308 ]]
5,77,[0.73351 0.82687],[0.8981],[[1.30132 1.4776 ]],[[0.2 0.45625 0.55625] [0.5 0.7318 0.8318 ]]
6,78,[0.73397 0.82715],[0.89871],[[1.30474 1.48145]],[[0.2 0.45744 0.55744] [0.5 0.73278 0.83278]]
7,79,[0.73443 0.82743],[0.89932],[[1.30812 1.48527]],[[0.2 0.45861 0.55861] [0.5 0.73376 0.83376]]
8,80,[0.73489 0.82771],[0.89991],[[1.31147 1.48904]],[[0.2 0.45978 0.55978] [0.5 0.73473 0.83473]]
9,81,[0.73533 0.82798],[0.9005],[[1.31478 1.49277]],[[0.2 0.46093 0.56093] [0.5 0.73568 0.83568]]




inputs: [1, 1, 1]; target_outputs: [0]


,1.step,2.actual_output_of_hus,2.actual_output_of_ous,3.weights_to_ous,4.weights_to_hus
0,109,[0.75053 0.88578],[0.10616],[[-1.09062 -1.4812 ]],[[0.26714 0.36714 0.46714] [0.58279 0.68279 0.78279]]
1,110,[0.7511 0.88601],[0.10543],[[-1.0944 -1.48567]],[[0.26817 0.36817 0.46817] [0.58355 0.68355 0.78355]]
2,111,[0.75167 0.88624],[0.10471],[[-1.09814 -1.49007]],[[0.26919 0.36919 0.46919] [0.58429 0.68429 0.78429]]
3,112,[0.75224 0.88646],[0.104],[[-1.10183 -1.49442]],[[0.27019 0.37019 0.47019] [0.58503 0.68503 0.78503]]
4,113,[0.75279 0.88668],[0.1033],[[-1.10547 -1.49872]],[[0.27119 0.37119 0.47119] [0.58576 0.68576 0.78576]]
5,114,[0.75334 0.8869 ],[0.10262],[[-1.10907 -1.50296]],[[0.27217 0.37217 0.47217] [0.58648 0.68648 0.78648]]
6,115,[0.75388 0.88711],[0.10194],[[-1.11263 -1.50715]],[[0.27315 0.37315 0.47315] [0.58719 0.68719 0.78719]]
7,116,[0.75442 0.88732],[0.10128],[[-1.11615 -1.51129]],[[0.27411 0.37411 0.47411] [0.58789 0.68789 0.78789]]
8,117,[0.75495 0.88753],[0.10063],[[-1.11963 -1.51538]],[[0.27506 0.37506 0.47506] [0.58859 0.68859 0.78859]]
9,118,[0.75547 0.88774],[0.09999],[[-1.12307 -1.51942]],[[0.27601 0.37601 0.47601] [0.58928 0.68928 0.78928]]


{'inputs_list': [[1, 0]],
 'learning_coef': 0.5,
 'lower_margin': 0.1,
 'target_outputs_list': [[1, 0]],
 'upper_margin': 0.9,
 'weights_to_hidden_units': [[0.2, 0.5], [0.3, 0.6], [0.4, 0.7]],
 'weights_to_output_units': [[0.8, 0.9, 0.1], [0.25, 0.35, 0.15]]}


inputs: [1, 0]; target_outputs: [1, 0]


,1.step,2.actual_output_of_hus,2.actual_output_of_ous,3.weights_to_ous,4.weights_to_hus
0,100,[0.6521 0.6694 0.67654],[0.91265 0.10605],[[ 1.35937 1.47876 0.69495] [-1.0224 -0.96801 -1.20767]],[[0.6283 0.5 ] [0.70545 0.6 ] [0.73791 0.7 ]]
1,101,[0.65261 0.66989 0.67695],[0.91316 0.10528],[[ 1.36164 1.48109 0.6973 ] [-1.02567 -0.97138 -1.21107]],[[0.63054 0.5 ] [0.70767 0.6 ] [0.73977 0.7 ]]
2,102,[0.65311 0.67037 0.67735],[0.91365 0.10452],[[ 1.36389 1.48339 0.69963] [-1.02891 -0.9747 -1.21443]],[[0.63275 0.5 ] [0.70986 0.6 ] [0.74161 0.7 ]]
3,103,[0.65361 0.67085 0.67775],[0.91414 0.10377],[[ 1.36611 1.48568 0.70194] [-1.0321 -0.97798 -1.21774]],[[0.63495 0.5 ] [0.71203 0.6 ] [0.74343 0.7 ]]
4,104,[0.6541 0.67132 0.67814],[0.91463 0.10304],[[ 1.36831 1.48794 0.70422] [-1.03526 -0.98122 -1.22101]],[[0.63712 0.5 ] [0.71418 0.6 ] [0.74523 0.7 ]]
5,105,[0.65459 0.67179 0.67853],[0.9151 0.10232],[[ 1.37049 1.49017 0.70648] [-1.03837 -0.98441 -1.22424]],[[0.63926 0.5 ] [0.71631 0.6 ] [0.74701 0.7 ]]
6,106,[0.65507 0.67226 0.67891],[0.91557 0.10161],[[ 1.37265 1.49239 0.70872] [-1.04145 -0.98757 -1.22743]],[[0.64139 0.5 ] [0.71841 0.6 ] [0.74877 0.7 ]]
7,107,[0.65554 0.67272 0.67929],[0.91603 0.10092],[[ 1.37479 1.49458 0.71094] [-1.04449 -0.99069 -1.23058]],[[0.64349 0.5 ] [0.72049 0.6 ] [0.75052 0.7 ]]
8,108,[0.65601 0.67317 0.67967],[0.91649 0.10023],[[ 1.37691 1.49676 0.71313] [-1.04749 -0.99377 -1.23369]],[[0.64557 0.5 ] [0.72255 0.6 ] [0.75225 0.7 ]]
9,109,[0.65648 0.67362 0.68004],[0.91693 0.09956],[[ 1.379 1.49891 0.7153 ] [-1.05045 -0.99681 -1.23676]],[[0.64764 0.5 ] [0.72459 0.6 ] [0.75396 0.7 ]]


In [17]:
def bpa_out_to_in(dataset, num_epochs = 1, roundup = 4):
    log = []

    epoch = 0
    weights_to_hus = copy.deepcopy(dataset['weights_to_hidden_units'])
    weights_to_ous = copy.deepcopy(dataset['weights_to_output_units'])
    target_outputs = copy.deepcopy(dataset['target_outputs_list'])

    while epoch <= num_epochs:
        actual_outputs_of_ous = []*len(target_outputs)
        actual_outputs_of_hus = []*len(target_outputs)
        
        for i, inputs in enumerate(dataset['inputs_list']):
            actual_output_of_hus = []
            for weights_to_hu in weights_to_hus:
                actual_output_of_hus.append(actual_output(weighted_sum(inputs, weights_to_hu)))
            
            actual_output_of_ous = []
            for weights_to_ou in weights_to_ous:
                actual_output_of_ous.append(actual_output(weighted_sum(actual_output_of_hus, weights_to_ou)))

            temp_weights_to_ous = copy.deepcopy(weights_to_ous)
            delta_ous = []
            for k, actual_output_of_ou in enumerate(actual_output_of_ous):
                delta_ous.append(delta_ou(target_outputs[i][k], actual_output_of_ou))
                for j, actual_output_of_hu in enumerate(actual_output_of_hus):
                    weights_to_ous[k][j] = new_weight_to_ou(weights_to_ous[k][j], dataset['learning_coef'], delta_ous[k], actual_output_of_hu)

            # print(f'delta_ous: {delta_ous}')
            # print(f'weights_to_ous: {weights_to_ous}')

            delta_hus = []
            for j, actual_output_of_hu in enumerate(actual_output_of_hus):
                temp_delta_hu = 0
                for k, actual_output_of_ou in enumerate(actual_output_of_ous):
                    temp_delta_hu += delta_hu(delta_ous[k], temp_weights_to_ous[k][j], actual_output_of_hu)
                delta_hus.append(temp_delta_hu)

            for j, actual_output_of_hu in enumerate(actual_output_of_hus):
                for i, input in enumerate(inputs):
                    weights_to_hus[j][i] = new_weight_to_hu(weights_to_hus[j][i], dataset['learning_coef'], delta_hus[j], input)
            
            # print(f'delta_hus: {delta_hus}')
            # print(f'weights_to_hus: {weights_to_hus}')

            actual_outputs_of_ous.append(actual_output_of_ous)
            actual_outputs_of_hus.append(actual_output_of_hus)
        
        sub_log = {}
        sub_log.update({"1.epoch": epoch})
        # sub_log.update({"2.inputs": np.round(inputs, roundup)})
        sub_log.update({"2.target_outputs": np.round(target_outputs, roundup)})
        # sub_log.update({"2.actual_outputs_of_hus": np.round(actual_outputs_of_hus, roundup)})
        sub_log.update({"2.actual_outputs": np.round(actual_outputs_of_ous, roundup)})
        sub_log.update({"3.weights_to_output_units": np.round(weights_to_ous, roundup)})
        sub_log.update({"4.weights_to_hidden_units": np.round(weights_to_hus, roundup)})
        log.append(sub_log)

        
        if terminate_training(actual_output_of_ous, target_outputs, dataset['lower_margin'], dataset['upper_margin']): break
        
        epoch += 1
    return log

In [20]:
for data in dataset:
    pp.pprint(data)
    log = bpa_out_to_in(data, 100000, 5)
    # df = pd.DataFrame(log[:10])
    df = pd.DataFrame(log[-5:])
    # df = pd.DataFrame([log])
    display(df.style)
    break

{'inputs_list': [[0, 0, 0],
                 [1, 0, 0],
                 [0, 1, 0],
                 [1, 1, 0],
                 [0, 0, 1],
                 [1, 0, 1],
                 [0, 1, 1],
                 [1, 1, 1]],
 'learning_coef': 0.5,
 'lower_margin': 0.1,
 'target_outputs_list': [[0], [1], [1], [0], [0], [1], [1], [0]],
 'upper_margin': 0.9,
 'weights_to_hidden_units': [[0.2, 0.3, 0.4], [0.5, 0.6, 0.7]],
 'weights_to_output_units': [[0.8, 0.9]]}


,1.epoch,2.target_outputs,2.actual_outputs_of_ous,3.weights_to_ous
0,99996,[[0] [1] [1] [0] [0] [1] [1] [0]],[[0.02772] [0.88738] [0.88702] [0.52534] [0.00095] [0.93871] [0.93912] [0.53398]],[[-53.97289 46.85773]]
1,99997,[[0] [1] [1] [0] [0] [1] [1] [0]],[[0.02772] [0.88738] [0.88702] [0.52534] [0.00095] [0.93871] [0.93912] [0.53398]],[[-53.97308 46.85791]]
2,99998,[[0] [1] [1] [0] [0] [1] [1] [0]],[[0.02772] [0.88738] [0.88702] [0.52534] [0.00095] [0.93871] [0.93912] [0.53398]],[[-53.97326 46.85809]]
3,99999,[[0] [1] [1] [0] [0] [1] [1] [0]],[[0.02772] [0.88738] [0.88702] [0.52534] [0.00095] [0.93871] [0.93912] [0.53398]],[[-53.97345 46.85827]]
4,100000,[[0] [1] [1] [0] [0] [1] [1] [0]],[[0.02772] [0.88738] [0.88702] [0.52534] [0.00095] [0.93871] [0.93912] [0.53398]],[[-53.97364 46.85845]]
